<a href="https://colab.research.google.com/github/Ashi743/pytorch/blob/main/dataset_and_dataloader_class_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from sklearn.datasets import make_classification
import torch
import numpy as np

In [33]:
X,y= make_classification(
    n_samples=100,
    n_features=2,
    n_informative=2,
    n_redundant=0,
    random_state=1,
    n_classes=2,
)

In [34]:
X.shape,y.shape

((100, 2), (100,))

In [35]:
#numpy to tensors
X=torch.tensor(X ,dtype=torch.float32)
y=torch.tensor(y, dtype=torch.float32)

In [11]:
from torch.utils.data import TensorDataset,DataLoader, Dataset

In [36]:
class CustomDataset(Dataset):
  def __init__(self, features, labels):
    self.features = features
    self.labels = labels

  def __len__(self):
    return self.features.shape[0]

  def __getitem__(self,index):
    return self.features[index], self.labels[index]             #ANY TRANSFORMATIONS CAN AND WILL BE DONE HERE



In [37]:
dataset= CustomDataset(X,y)

In [38]:
len(dataset)

100

In [39]:
dataset[4]

(tensor([-1.2722,  0.2595]), tensor(0.))

In [15]:
from inspect import isgenerator
isgenerator(dataset)

False

In [17]:
# Check if it's iterable
from collections.abc import Iterable
print(isinstance(dataset, Iterable))

# Check if it's an iterator
from collections.abc import Iterator
isinstance(dataset, Iterator)


False


False

In [46]:
dataloader= DataLoader(dataset, batch_size=2, shuffle= True, num_workers=4)   # AND THATS IT   #num worker leads to parrallelizations

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [22]:
# Check if it's iterable
from collections.abc import Iterable
print(isinstance(dataloader, Iterable))

# Check if it's an iterator
from collections.abc import Iterator
isinstance(dataloader, Iterator)


True


False

In [47]:
for batch_features, batch_labels in dataloader:
  print(batch_features)
  print(batch_labels)
  print("--+++---"*20)

tensor([[-1.5440,  0.0119],
        [ 1.4418, -0.5601]])
tensor([0., 1.])
--+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++---
tensor([[-0.7073, -1.7515],
        [-1.0722, -0.7702]])
tensor([0., 0.])
--+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++---
tensor([[ 2.1496, -0.3115],
        [-1.5159, -0.0249]])
tensor([1., 0.])
--+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++---
tensor([[ 0.7099,  0.6902],
        [-0.8479, -1.3662]])
tensor([1., 0.])
--+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++-----+++---
tensor([[-1.0102,  0.8581],
        [-1.5692, -0.5405]])
ten

**TO USE ABOVE IN BREAST CANCER DATASET**

In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from sklearn.preprocessing import StandardScaler, LabelEncoder
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [51]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.drop('Unnamed: 32', axis=1, inplace=True)
df.drop('id', axis=1, inplace=True)
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [52]:
X_train,X_test ,y_train, y_test= train_test_split(df.drop('diagnosis', axis=1), df['diagnosis'], test_size=0.2, random_state=42)

In [53]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [54]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [55]:
#numpy to torch
X_train= torch.tensor(X_train, dtype=torch.float32)
X_test= torch.tensor(X_test, dtype=torch.float32)
y_train= torch.tensor(y_train, dtype=torch.float32)
y_test= torch.tensor(y_test, dtype=torch.float32)

In [56]:
X_train.shape, y_train.shape

(torch.Size([455, 30]), torch.Size([455]))

In [85]:
from torch.utils.data import Dataset,DataLoader

class customDataset(Dataset):
  def __init__(self, features, labels):
    self.features= features
    self.labels= labels

  def __len__(self):
    return self.features.shape[0]

  def __getitem__(self, index):
    return self.features[index], self.labels[index]

dataset_train= customDataset(X_train, y_train)
dataset_test= customDataset(X_test, y_test)
dataloader_train= DataLoader(dataset_train, batch_size=32, shuffle=True)
dataloader_test= DataLoader(dataset_test, batch_size=32, shuffle=True)

In [86]:
#define model
import torch.nn as nn
class breastCancerModel(nn.Module):
  def __init__(self, num_features):
    super().__init__()
    self.fc1= nn.Linear(num_features,1)

    self.fc2= nn.Sigmoid()

  def forward(self, x):
    x= self.fc1(x)

    x= self.fc2(x)
    return x

In [87]:
#important params
epochs= 25
learning_rate= 0.1


In [88]:
#create model
model= breastCancerModel(num_features=X_train.shape[1])

#optimizer
optimizer= torch.optim.Adam(model.parameters(), lr=learning_rate)

#loss
loss_fn= nn.BCELoss()

In [89]:
#training pipeline
#define loop
#for epoch
for epoch in range(epochs):
  #for batch
  for batch_features, batch_labels in dataloader_train:
    #forward pass
    outputs= model(batch_features)
    #calculate loss
    loss= loss_fn(outputs, batch_labels.unsqueeze(1))
    #clear gradient
    optimizer.zero_grad()
    #backward pass
    loss.backward()
    #update weights
    optimizer.step()

  print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

Epoch 1/25, Loss: 0.20781835913658142
Epoch 2/25, Loss: 0.010034648701548576
Epoch 3/25, Loss: 0.0008498672978021204
Epoch 4/25, Loss: 0.005264059640467167
Epoch 5/25, Loss: 0.1661059707403183
Epoch 6/25, Loss: 0.011050092056393623
Epoch 7/25, Loss: 0.051091838628053665
Epoch 8/25, Loss: 0.0006627390976063907
Epoch 9/25, Loss: 0.14168651401996613
Epoch 10/25, Loss: 0.004091504495590925
Epoch 11/25, Loss: 4.097621058463119e-05
Epoch 12/25, Loss: 0.00014871424355078489
Epoch 13/25, Loss: 0.00014578676200471818
Epoch 14/25, Loss: 0.0711902379989624
Epoch 15/25, Loss: 0.002475793706253171
Epoch 16/25, Loss: 0.031441498547792435
Epoch 17/25, Loss: 0.2678547203540802
Epoch 18/25, Loss: 0.0022286383900791407
Epoch 19/25, Loss: 0.0022554227616637945
Epoch 20/25, Loss: 0.008897929452359676
Epoch 21/25, Loss: 0.001208786154165864
Epoch 22/25, Loss: 0.0009823994478210807
Epoch 23/25, Loss: 0.24260897934436798
Epoch 24/25, Loss: 0.00020989238691981882
Epoch 25/25, Loss: 0.00035805031075142324


In [90]:
#evaluation
model.eval()
accuracy_list=[]
with torch.no_grad():
  for batch_features, batch_labels in dataloader_test:
    outputs= model(batch_features)
    predicted= torch.round(outputs)

    accuracy= (predicted==batch_labels.unsqueeze(1)).float().mean()
    accuracy_list.append(accuracy.item())

print(f"Accuracy: {sum(accuracy_list)/len(accuracy_list)}")

Accuracy: 0.984375
